DDR Midterm - Haonan Wang

In [1]:
# import packages
import requests
from bs4 import BeautifulSoup
import time
import random
import re
import os
import json
import pymysql
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd

# Part 1: SHORT, LET GET STARTED

Go to http://numbersapi.com and familiarize yourself with the API.

a) Write a program that accesses all trivia results for the numbers from 0 (zero) to 99 using batch requests only (One single query for all the numbers). Print the output result to the screen in the format of [3-digit number with leading zeros] - [TRIVIA] (One number per each line)

Note: your output for each number may be different. The website randomizes its trivia responses.

In [3]:
# create headers to help enter the website

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}

In [4]:
# loop to get content

for i in range(0,100):
    n = str(i)
    url = 'http://numbersapi.com/' + n
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    print(n.zfill(3), '-', soup)

000 - 0 is the coldest possible temperature old the Kelvin scale.
001 - 1 is the number of dimensions of a line.
002 - 2 is the first magic number in physics.
003 - 3 is the number of witches in William Shakespeare's Macbeth.
004 - 4 is the number of strings on a violin, a viola, a cello, double bass, a cuatro and a ukulele, and the number of string pairs on a mandolin.
005 - 5 is the holy number of Discordianism, as dictated by the Law of Fives.
006 - 6 is the number worn by LeBron James in the NBA.
007 - 7 is the number of main stars in the constellations of the Big Dipper and Orion.
008 - 8 is the number of legs that arachnids have.
009 - 9 is the number of circles of Hell in Dante's Divine Comedy.
010 - 10 is the average thickness of the Arctic ice sheet in feet.
011 - 11 is the number of pounds one gallon of pure maple syrup weighs.
012 - 12 is the number of ounces in a troy pound (used for precious metals).
013 - 13 is the number of steps in the gallows.
014 - 14 is the earliest 

099 - 99 is the number of names of Allah, the names or attributes of God in the Qur'an.


# Part 2: LONGER!

## Web-scraping: Which sellers advertise/sponsor on eBay

### a

a) Take the code your group developed for assignment 2 and fix everything that may not have been optimal in your previous solution. Alternatively, but less cool, you may use our solution code as your starting point. (You will use this for the next part. You do not need to include this code in your submission.)

How would you modify your search query to only include buy-it-now (non-auction) items? 

What is the GET request's variable name corresponding to buy-it-now searches? 

How would you modify your search query to include 100 items per search result page? 

What is the GET request's variable name corresponding to items per page searches? 

Include your answers as a comment in your code marking it as Part 2-a. (This section does not involve coding)

In [ ]:
# Add LH_BIN=1 to the url. https://www.ebay.com/sch/i.html?_nkw=samsung+tv&LH_BIN=1
# LH_BIN
# Add _ipg=100 to the url. https://www.ebay.com/sch/i.html?_nkw=samsung+tv&_ipg=100
# _ipg

### b

b) Modify your previous code to search for buy-it-now listings of <keyword> and limit the number of items to 100 per page. Use this guide to select the right <keyword> assigned to you:

If your last name on Canvas starts with A-F use <keyword>: "canon eos 5d"

If your last name on Canvas starts with G-L use <keyword>: "playstation 4 slim"

If your last name on Canvas starts with M-R use <keyword>: "iphone 11 pro"

If your last name on Canvas starts with S-Z use <keyword>: "ipad 4"

Identify all the sponsored items.

For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)

In [5]:
# create headers to enter the website

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}

In [6]:
# download and save the first 10 pages of results.(with a random interval between 10 and 20 seconds to mimic human behavior)

for i in range(1, 11):
    n = str(i)
    url = 'https://www.ebay.com/sch/i.html?_nkw=ipad+4&LH_BIN=1&_ipg=100&_pgn='+n.zfill(2)
    page = requests.get(url, headers=headers)
    search = open('ebay_ipad_4_'+n.zfill(2)+'.htm', 'w', encoding = 'utf-8')
    search.write(page.text)
    search.close()
    time.sleep(random.randint(10,20));

In [7]:
# parse the pages and save sponsored and unsponsored items respectively

sponsored = open('sponsored.txt', 'w', encoding = 'utf-8')
non_sponsored = open('non-sponsored.txt', 'w', encoding = 'utf-8')

for i in range(1, 11):
    n = str(i)
    page_result = open('ebay_ipad_4_'+n.zfill(2)+'.htm', 'r', encoding = "utf-8")
    text = page_result.read()
    soup = BeautifulSoup(text,'html.parser')
    products = soup.select("#srp-river-results > ul > li.s-item")
    for product in products:
        a = product.find("a")
        if not a:
            continue 
        text_span = product.find("span").get_text()
        if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*', text_span):
            url_simplify_s = re.sub("(.*)\\?.*", r"\1", a['href'])
            #sponsored.write('Page'+n.zfill(2)+' '+url_simplify_s+'\n')
            sponsored.write(url_simplify_s+'\n')

        else:
            url_simplify_ns = re.sub("(.*)\\?.*", r"\1", a['href'])
            #non_sponsored.write('Page'+n.zfill(2)+' '+url_simplify_ns+'\n');
            non_sponsored.write(url_simplify_ns+'\n');

sponsored.close()
non_sponsored.close()

### c

c) Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "<item-id>.htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [8]:
# create two new folders

os.mkdir("sponsored")
os.mkdir("non-sponsored")

In [99]:
# download all urls for sponsored items and save them in 'sponsored' folder

sponsored = open('sponsored.txt', 'r', encoding = 'utf-8')
spon_lines = sponsored.readlines()
os.chdir('C:/Users/whn97/Desktop/422_Data Design & Representation_BoehnkeJorn/Midterm/sponsored')
for spon_line in spon_lines:
    path = spon_line.split('\n')[0]
    itemid = path.split('/')[5]
    page = requests.get(path, headers=headers)
    search = open(itemid+'.htm', 'w', encoding = 'utf-8')
    search.write(page.text)
    search.close()
    time.sleep(2);

In [3]:
# save id for all sponsored items

id_spon=[]
for spon_line in spon_lines:
    path = spon_line.split('\n')[0]
    itemid = path.split('/')[5]
    id_spon.append(itemid)

In [4]:
# check the number of id

print(len(id_spon))

72


In [9]:
# go back to the parent path

os.chdir('C:/Users/whn97/Desktop/422_Data Design & Representation_BoehnkeJorn/Midterm')

In [12]:
# download all urls for non-sponsored items and save them in 'non-sponsored' folder

non_sponsored = open('non-sponsored.txt', 'r', encoding = 'utf-8')
non_lines = non_sponsored.readlines()
id_non=[]
os.chdir('C:/Users/whn97/Desktop/422_Data Design & Representation_BoehnkeJorn/Midterm/non-sponsored')
for non_line in non_lines:
    path = non_line.split('\n')[0]
    itemid = path.split('/')[5]
    id_non.append(itemid)
    page = requests.get(path, headers=headers)
    search = open(itemid+'.htm', 'w', encoding = 'utf-8')
    search.write(page.text)
    search.close()
    time.sleep(2);

In [6]:
# save id for all non-sponsored items

id_non=[]
for non_line in non_lines:
    path = non_line.split('\n')[0]
    itemid = path.split('/')[5]
    id_non.append(itemid)

In [7]:
# check the number of id

print(len(id_non))

1034


In [13]:
# go back to the parent path

os.chdir('C:/Users/whn97/Desktop/422_Data Design & Representation_BoehnkeJorn/Midterm')

### d

d) Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:

seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In your code, highlight the selector command you choose to obtain each element using comments.

In [8]:
# go to the 'sponsored' folder

os.chdir('C:/Users/whn97/Desktop/422_Data Design & Representation_BoehnkeJorn/Midterm/sponsored')

In [9]:
# create lists to store information

seller_name_spon=[]
seller_score_spon=[]
item_price_spon=[]
item_sold_spon=[]
best_offer_available_spon=[]
title_spon=[]
returns_allowed_spon=[]
shipping_price_spon=[]
condition_spon=[]
shipping_computed_spon=[]

sponsored_or_not=[]

In [10]:
# get required information about sponsored items

n = len(id_spon)
for i in range(0,n):
    page = open(id_spon[i]+'.htm', 'r', encoding = 'utf-8')
    text = page.read()
    soup = BeautifulSoup(text,'html.parser')
    
    seller = soup.find(id='RightSummaryPanel')
    
    # seller name
    seller_name = seller.find('span', class_='mbg-nw')
    seller_name_spon.append(seller_name.text)
    
    # seller score
    seller_score = seller.find('span', class_='mbg-l')
    score = seller_score.find('a')
    seller_score_spon.append(score.text)
    
    item = soup.find(id='LeftSummaryPanel')
    
    # item price
    if soup.find(id='prcIsum'):
        item_price = soup.find(id='prcIsum')
        item_price_spon.append(item_price.text)
    elif soup.find(id='mm-saleDscPrc'):
        item_price = soup.find(id='mm-saleDscPrc')
        item_price_spon.append(item_price.text)
    else:
        item_price_spon.append('null')
    
    # item sold
    if item.find('a', class_='vi-txt-underline'):
        item_sold = item.find('a', class_='vi-txt-underline')
        item_sold_spon.append(item_sold.text)
    else:
        item_sold_spon.append('null')
    
    # best offer
    if item.find('div', class_='vi-bbox-dspn u-flL lable boLable'):
        best_offer_available_spon.append('true')
    else:
        best_offer_available_spon.append('false')
        
    # item title
    title = item.find(id='itemTitle')
    title_spon.append(title.text)
    
    # returns allowed
    if soup.find(id='vi-ret-accrd-txt'):
        returns = soup.find(id='vi-ret-accrd-txt')
        returns_allowed_spon.append(returns.text)
    else:
        returns_allowed_spon.append('null')
        
    # shipping price
    shipping = soup.find(id='shSummary')
    if shipping.find(id='fshippingCost'):
        cost = shipping.find(id='fshippingCost')
        price = cost.find('span')
        shipping_price_spon.append(price.text)
        shipping_computed_spon.append('true')
    elif shipping.find('strong'):                  # free n 
        price = shipping.find('strong')
        shipping_price_spon.append(price.text)
        shipping_computed_spon.append('true')
    elif shipping.find(id='e3'):                   # calculate
        price = shipping.find(id='e3')
        shipping_price_spon.append(price.text)
        shipping_computed_spon.append('false')
    else:
        price = 'null'
        shipping_price_spon.append(price)
        shipping_computed_spon.append('false')
    
    # condition
    condition = item.find(id='vi-itm-cond')
    condition_spon.append(condition.text)
    
    # sponsored or not
    sponsored_or_not.append('true');
    

In [220]:
#print(seller_name_spon)
#print(seller_score_spon)
#print(item_price_spon)
#print(item_sold_spon)
#print(best_offer_available_spon)
#print(title_spon)
#print(returns_allowed_spon)
#print(shipping_price_spon)
#print(condition_spon)
#print(shipping_computed_spon)
#print(sponsored_or_not)
#id_spon[1]

In [11]:
# go to the 'non-sponsored' folder

os.chdir('C:/Users/whn97/Desktop/422_Data Design & Representation_BoehnkeJorn/Midterm/non-sponsored')

In [12]:
# create lists to store information

seller_name_non=[]
seller_score_non=[]
item_price_non=[]
item_sold_non=[]
best_offer_available_non=[]
title_non=[]
returns_allowed_non=[]
shipping_price_non=[]
condition_non=[]
shipping_computed_non=[]

In [13]:
# get required information about non-sponsored items

ns = len(id_non)
for i in range(0,ns):
    page = open(id_non[i]+'.htm', 'r', encoding = 'utf-8')
    text = page.read()
    soup = BeautifulSoup(text,'html.parser')
    
    seller = soup.find(id='RightSummaryPanel')
    
    # seller name
    seller_name = seller.find('span', class_='mbg-nw')
    seller_name_non.append(seller_name.text)
    
    # seller score
    seller_score = seller.find('span', class_='mbg-l')
    score = seller_score.find('a')
    seller_score_non.append(score.text)
    
    item = soup.find(id='LeftSummaryPanel')
    
    # item price
    if soup.find(id='prcIsum'):
        item_price = soup.find(id='prcIsum')
        item_price_non.append(item_price.text)
    elif soup.find(id='mm-saleDscPrc'):
        item_price = soup.find(id='mm-saleDscPrc')
        item_price_non.append(item_price.text)
    else:
        item_price_non.append('null')
    
    # item sold
    if item.find('a', class_='vi-txt-underline'):
        item_sold = item.find('a', class_='vi-txt-underline')
        item_sold_non.append(item_sold.text)
    else:
        item_sold_non.append('null')
    
    # best offer
    if item.find('div', class_='vi-bbox-dspn u-flL lable boLable'):
        best_offer_available_non.append('true')
    else:
        best_offer_available_non.append('false')
        
    # item title
    title = item.find(id='itemTitle')
    title_non.append(title.text)
    
    # returns allowed
    if soup.find(id='vi-ret-accrd-txt'):
        returns = soup.find(id='vi-ret-accrd-txt')
        returns_allowed_non.append(returns.text)
    else:
        returns_allowed_non.append('null')
        
    # shipping price
    shipping = soup.find(id='shSummary')
    if shipping.find(id='fshippingCost'):
        cost = shipping.find(id='fshippingCost')
        price = cost.find('span')
        shipping_price_non.append(price.text)
        shipping_computed_non.append('true')
    elif shipping.find('strong'):                  # free n 
        price = shipping.find('strong')
        shipping_price_non.append(price.text)
        shipping_computed_non.append('true')
    elif shipping.find(id='e3'):                   # calculate
        price = shipping.find(id='e3')
        shipping_price_non.append(price.text)
        shipping_computed_non.append('false')
    else:
        price = 'null'
        shipping_price_non.append(price)
        shipping_computed_non.append('false')
    
    # condition
    condition = item.find(id='vi-itm-cond')
    condition_non.append(condition.text)
    
    # sponsored or not
    sponsored_or_not.append('false');
    

In [219]:
#print(seller_name_non)
#print(seller_score_non)
#print(item_price_non)
#print(item_sold_non)
#print(best_offer_available_non)
#print(title_non)
#print(returns_allowed_non)
#print(shipping_price_non)
#print(condition_non)
#print(shipping_computed_non)
#print(sponsored_or_not)
#id_non[74]

In [14]:
# combine results of sponsored and non-sponsored items

id_spon_copy = id_spon.copy()
id_non_copy = id_non.copy()
item_id = id_spon_copy + id_non_copy
seller_name = seller_name_spon + seller_name_non
seller_score = seller_score_spon + seller_score_non
item_price = item_price_spon + item_price_non
item_sold = item_sold_spon + item_sold_non
best_offer_available = best_offer_available_spon + best_offer_available_non
title = title_spon + title_non
returns_allowed = returns_allowed_spon + returns_allowed_non
shipping_price = shipping_price_spon + shipping_price_non
condition = condition_spon + condition_non
shipping_computed = shipping_computed_spon + shipping_computed_non

In [15]:
# check the number of lines of information about all items

print(len(item_id))
print(len(seller_name))
print(len(seller_score))
print(len(item_price))
print(len(item_sold))
print(len(best_offer_available))
print(len(title))
print(len(returns_allowed))
print(len(shipping_price))
print(len(condition))
print(len(shipping_computed))
print(len(sponsored_or_not))

1106
1106
1106
1106
1106
1106
1106
1106
1106
1106
1106
1106


### e

e) Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Create a database and name it "eBay". Save the information of items in (d)  into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.  If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [16]:
# connect to mysql

connection = pymysql.connect(host='localhost', user='root')
mydb = connection.get_server_info()
print("Connected to MySQL Server version ", mydb)

Connected to MySQL Server version  8.0.17


In [17]:
# create 'eBay' database

mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE if not exists eBay")
sql_db_select = "use eBay;"
mycursor.execute(sql_db_select)

# create 'eBay_items' table

create_table = "CREATE TABLE eBay_items (item_id VARCHAR(100), seller_name VARCHAR(255), seller_score INT, item_price INT, currency VARCHAR(100), item_sold INT, best_offer_available BOOLEAN, title VARCHAR(255), returns_allowed BOOLEAN, shipping_price INT, item_condition VARCHAR(255), shipping_computed BOOLEAN, sponsored_or_not BOOLEAN)"
mycursor.execute(create_table)

0

##### check the data type and form of each list

In [18]:
# split item price into item_price_amount and currency

price_amount=[]
currency=[]
item_price_amount=[]
len_price = len(item_price)
print(len_price)

for i in range(0, len_price):
    price_amount.append(item_price[i].split('$')[1])
    currency.append(item_price[i].split('$')[0]) 

for i in range(0, len_price):
    price = price_amount[i].split('/')[0]
    amount = price.replace(',', '')
    item_price_amount.append(float(amount)*100)

1106


In [19]:
# seller score

len_score = len(seller_score)
for i in range(0, len_score):
    score = seller_score[i].replace(',', '')
    seller_score[i] = int(score)

In [20]:
# item sold amount

item_sold_amount = []
len_sold = len(item_sold)
for i in range(0, len_sold):
    item_sold_amount.append(item_sold[i].split(' ')[0])
for i in range(0, len_sold):
    if re.search('null', item_sold_amount[i]):
        item_sold_amount[i] = None
    else:
        sold = item_sold_amount[i].replace(',', '')
        item_sold_amount[i] = int(sold)

In [21]:
# ship price amount

len_ship = len(shipping_price)
shipping_price_amount = []
ship_price_amount = []
for i in range(0, len_ship):
    if re.search(".*FREE.*", shipping_price[i]):
        shipping_price_amount.append('0')
    elif re.search("Calculate", shipping_price[i]):
        shipping_price_amount.append('null')
    elif re.search("null", shipping_price[i]):
        shipping_price_amount.append('null')
    else:
        amount = shipping_price[i].split('$')[1]
        shipping_price_amount.append(amount)
        
for i in range(0, len_ship):
    if re.search("null", shipping_price_amount[i]):
        ship_price_amount.append(None)
    else:
        ship_price_amount.append(float(shipping_price_amount[i])*100)

In [22]:
# return_allowed

return_allowed = []
len_re = len(returns_allowed)
for i in range(0, len_re):
    if re.search(".*not accept.*", returns_allowed[i]):
        return_allowed.append('false')
    else:
        return_allowed.append('true')

In [23]:
# item_title

len_t = len(title)
item_title=[]
for i in range(0, len_t):
    item = title[i].replace('Details about ', '')
    item_title.append(item)

##### insert table into mysql database

In [24]:
#Create engine to establish connection to mysql 

engine = create_engine("mysql+pymysql://{user}@localhost/{db}"
                       .format(user="root",
                               db="eBay"))

In [25]:
# create a dataframe to gather all information

ipad4 = pd.DataFrame({'item_id': item_id, 'seller_name': seller_name, 
                      'seller_score': seller_score, 'item_price': item_price_amount, 
                      'currency': currency, 'item_sold': item_sold_amount, 
                      'best_offer_available': best_offer_available, 'title': item_title, 
                      'returns_allowed': return_allowed, 'shipping_price': ship_price_amount, 
                      'item_condition': condition, 'shipping_computed': shipping_computed, 
                      'sponsored_or_not': sponsored_or_not})


In [26]:
ipad4.head()

,item_id,seller_name,seller_score,item_price,currency,item_sold,best_offer_available,title,returns_allowed,shipping_price,item_condition,shipping_computed,sponsored_or_not
0,293383390892,hawk_line,11373,10900.0,US,4.0,false,"Apple iPad 4 - 4th Generation 9.7"" with Reti...",true,0.0,Manufacturer refurbished,true,true
1,293452172205,hawk_line,11373,12000.0,US,NaN,false,"Apple iPad 4 - 4th Generation 9.7"" with Reti...",true,0.0,Manufacturer refurbished,true,true
2,172497427491,eazytrade84,84729,12985.0,US,518.0,false,"Apple iPad mini 2 16GB 32GB 64GB, WiFi + 4G ...",true,0.0,Manufacturer refurbished,true,true
3,202498264738,eazytrade84,84729,19985.0,US,797.0,false,Apple iPad Mini 4 WiFi + Cellular Unlocked I...,true,0.0,Seller refurbished,true,true
4,182386213481,eazytrade84,84729,11985.0,US,131.0,false,"Apple iPad 4th Gen Retina Display 32GB, Wi-F...",true,0.0,Manufacturer refurbished,true,true


In [27]:
# insert table into mysql

ipad4.to_sql(name = 'ebay_items', con = engine, if_exists = 'replace', index = False)

In [28]:
# Printing the data in ebay_items table

mycursor.execute("Select * from ebay_items")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['item_id','seller_name','seller_score','item_price','currency','item_sold','best_offer_available','title','returns_allowed','shipping_price','item_condition','shipping_computed','sponsored_or_not'])
print(result_df)

           item_id                seller_name  seller_score  item_price  \
0     293383390892                  hawk_line         11373     10900.0   
1     293452172205                  hawk_line         11373     12000.0   
2     172497427491                eazytrade84         84729     12985.0   
3     202498264738                eazytrade84         84729     19985.0   
4     182386213481                eazytrade84         84729     11985.0   
...            ...                        ...           ...         ...   
1101  183785311220    connectedsolutionsgroup          1811     18499.0   
1102  163459339261                orangecolby            55     36500.0   
1103  333378976277                trd_digital          3784     42900.0   
1104  153801170787           amazing-wireless        101991      9999.0   
1105  262783810753  certifiedrefurbishedstore         29575     13995.0   

     currency  item_sold best_offer_available  \
0         US         4.0                false   
1

### f

f) Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations. 

In [29]:
# seller name

mycursor.execute("select sponsored_or_not, item_condition, seller_name, count(seller_name) as count_seller from ebay_items group by sponsored_or_not, item_condition, seller_name;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'seller name', 'count'])
print(result_df)

    sponsored                 condition          seller name  count
0        true  Manufacturer refurbished            hawk_line      2
1        true  Manufacturer refurbished          eazytrade84     44
2        true        Seller refurbished          eazytrade84      1
3        true  Manufacturer refurbished              buyspry      2
4        true  Manufacturer refurbished     amazing-wireless      1
..        ...                       ...                  ...    ...
504     false  Manufacturer refurbished  palafoxcomputers850      1
505     false        Seller refurbished             cars4uma      1
506     false                       New         ktcorp2016_3      1
507     false                       New          orangecolby      1
508     false                  Open box     amazing-wireless      1

[509 rows x 4 columns]


In [30]:
# seller score

mycursor.execute("select sponsored_or_not, item_condition, avg(seller_score) as mean, min(seller_score) as minimun, max(seller_score) as maximum, std(seller_score) as standard_dev from ebay_items group by sponsored_or_not, item_condition;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'mean', 'min', 'max', 'std'])
print(result_df)

  sponsored                 condition         mean   min      max  \
0      true  Manufacturer refurbished   84642.6800  5297   187012   
1      true        Seller refurbished  116352.7000   837   213355   
2      true                      Used   72482.2500  5003   213355   
3      true                  Open box   30417.0000   308    84729   
4      true                       New    2716.0000  2716     2716   
5     false        Seller refurbished  154221.9265     3  2265016   
6     false                      Used   17844.5879     0   311735   
7     false                  Open box   33498.6458     0   213351   
8     false  Manufacturer refurbished  129344.5472     0  2265017   
9     false                       New    8064.1404     0   187013   

             std  
0   27674.131011  
1   99722.310289  
2   84601.896747  
3   38479.996682  
4       0.000000  
5  415645.903408  
6   46665.902562  
7   54300.001043  
8  208990.714522  
9   25114.912236  


In [31]:
# item price

mycursor.execute("select sponsored_or_not, item_condition, avg(item_price) as mean, min(item_price) as minimun, max(item_price) as maximum, std(item_price) as standard_dev from ebay_items group by sponsored_or_not, item_condition;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'mean', 'min', 'max', 'std'])
print(result_df)

  sponsored                 condition           mean       min       max  \
0      true  Manufacturer refurbished   19473.600000    7985.0   92499.0   
1      true        Seller refurbished   19145.000000   11995.0   24500.0   
2      true                      Used   13169.875000    5488.0   18985.0   
3      true                  Open box   35431.666667   19895.0   64900.0   
4      true                       New  129999.000000  129999.0  129999.0   
5     false        Seller refurbished   17302.647059    7995.0   99999.0   
6     false                      Used   16091.236443    3999.0   99999.0   
7     false                  Open box   22303.687500    8995.0   75000.0   
8     false  Manufacturer refurbished   21168.685535    5799.0   74299.0   
9     false                       New   50902.587719   14999.0  189999.0   

            std  
0  14307.429632  
1   4044.257039  
2   4330.008125  
3  20847.557944  
4      0.000000  
5  13272.401990  
6  11472.360530  
7  12360.618422  
8

In [32]:
# currency

mycursor.execute("select sponsored_or_not, item_condition, currency, count(currency) as count_curr from ebay_items group by sponsored_or_not, item_condition, currency;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'currency', 'count'])
print(result_df)

   sponsored                 condition currency  count
0       true  Manufacturer refurbished      US      50
1       true        Seller refurbished      US      10
2       true                      Used      US       8
3       true                  Open box      US       3
4       true                       New      US       1
5      false        Seller refurbished      US     204
6      false                      Used      US     460
7      false                  Open box      US      96
8      false  Manufacturer refurbished      US     159
9      false                      Used       C       1
10     false                       New      US     114


In [34]:
# item sold

mycursor.execute("select sponsored_or_not, item_condition, avg(item_sold) as mean, min(item_sold) as minimun, max(item_sold) as maximum, std(item_sold) as standard_dev from ebay_items where item_sold is not null group by sponsored_or_not, item_condition;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'mean', 'min', 'max', 'std'])
print(result_df)

  sponsored                 condition        mean    min      max          std
0      true  Manufacturer refurbished  105.666667    4.0    662.0   134.644336
1      true        Seller refurbished  241.250000    3.0    797.0   327.004874
2      true                      Used   67.000000    2.0    148.0    56.448206
3      true                  Open box  388.000000  388.0    388.0     0.000000
4     false        Seller refurbished  127.755319    1.0   2366.0   343.621906
5     false                      Used  133.600000    1.0   2076.0   372.031357
6     false                  Open box  267.882353    1.0   3294.0   772.741863
7     false  Manufacturer refurbished  330.452991    1.0  18670.0  1732.658308
8     false                       New    4.138889    1.0     17.0     4.035625


In [35]:
# best offer available

mycursor.execute("select sponsored_or_not, item_condition, best_offer_available, count(*) as count_best_offer from ebay_items group by sponsored_or_not, item_condition, best_offer_available;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'best offer', 'count'])
print(result_df)

   sponsored                 condition best offer  count
0       true  Manufacturer refurbished      false     50
1       true        Seller refurbished      false      9
2       true                      Used      false      8
3       true        Seller refurbished       true      1
4       true                  Open box      false      2
5       true                       New      false      1
6       true                  Open box       true      1
7      false        Seller refurbished      false    193
8      false                      Used      false    265
9      false                  Open box      false     90
10     false  Manufacturer refurbished      false    148
11     false                      Used       true    196
12     false                       New      false     88
13     false                  Open box       true      6
14     false                       New       true     26
15     false  Manufacturer refurbished       true     11
16     false        Seller refu

In [36]:
# title

mycursor.execute("select sponsored_or_not, item_condition, title, count(*) from ebay_items group by sponsored_or_not,item_condition, title;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'title', 'count'])
print(result_df)

    sponsored                 condition  \
0        true  Manufacturer refurbished   
1        true  Manufacturer refurbished   
2        true  Manufacturer refurbished   
3        true        Seller refurbished   
4        true  Manufacturer refurbished   
..        ...                       ...   
926     false                      Used   
927     false                       New   
928     false  Manufacturer refurbished   
929     false                  Open box   
930     false        Seller refurbished   

                                                 title  count  
0      Apple iPad 4 - 4th Generation 9.7" with Reti...      1  
1      Apple iPad 4 - 4th Generation 9.7" with Reti...      1  
2      Apple iPad mini 2 16GB 32GB 64GB, WiFi + 4G ...      1  
3      Apple iPad Mini 4 WiFi + Cellular Unlocked I...      1  
4      Apple iPad 4th Gen Retina Display 32GB, Wi-F...      1  
..                                                 ...    ...  
926    B-Stock - Unlocked Apple iPa

In [37]:
# returns allowed

mycursor.execute("select sponsored_or_not, item_condition, returns_allowed, count(*) as count_return from ebay_items group by sponsored_or_not, item_condition, returns_allowed;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'returns allowed', 'count'])
print(result_df)

   sponsored                 condition returns allowed  count
0       true  Manufacturer refurbished            true     50
1       true        Seller refurbished            true      9
2       true                      Used            true      8
3       true        Seller refurbished           false      1
4       true                  Open box            true      3
5       true                       New            true      1
6      false        Seller refurbished            true    193
7      false                      Used           false    223
8      false                      Used            true    238
9      false                  Open box            true     82
10     false  Manufacturer refurbished            true    147
11     false                  Open box           false     14
12     false                       New            true     69
13     false                       New           false     45
14     false  Manufacturer refurbished           false     12
15     f

In [38]:
# shipping_price

mycursor.execute("select sponsored_or_not, item_condition, avg(shipping_price) as mean, min(shipping_price) as minimun, max(shipping_price) as maximum, std(shipping_price) as standard_dev from ebay_items where shipping_price is not null group by sponsored_or_not, item_condition;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'mean', 'min', 'max', 'std'])
print(result_df)

  sponsored                 condition        mean  min     max         std
0      true  Manufacturer refurbished    0.000000  0.0     0.0    0.000000
1      true        Seller refurbished    0.000000  0.0     0.0    0.000000
2      true                      Used   61.875000  0.0   495.0  163.705862
3      true                  Open box    0.000000  0.0     0.0    0.000000
4      true                       New    0.000000  0.0     0.0    0.000000
5     false        Seller refurbished   47.238806  0.0  1995.0  261.801476
6     false                      Used  320.492401  0.0  4995.0  620.769623
7     false                  Open box   63.534091  0.0  1595.0  265.021652
8     false  Manufacturer refurbished   43.407895  0.0  4000.0  346.890290
9     false                       New  245.842105  0.0  2499.0  596.345296


In [39]:
# item condition

mycursor.execute("select sponsored_or_not, item_condition, count(*) from ebay_items group by sponsored_or_not, item_condition;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'count'])
print(result_df)

  sponsored                 condition  count
0      true  Manufacturer refurbished     50
1      true        Seller refurbished     10
2      true                      Used      8
3      true                  Open box      3
4      true                       New      1
5     false        Seller refurbished    204
6     false                      Used    461
7     false                  Open box     96
8     false  Manufacturer refurbished    159
9     false                       New    114


In [40]:
# shipping computed

mycursor.execute("select sponsored_or_not, item_condition, shipping_computed, count(*) as count from ebay_items group by sponsored_or_not, item_condition, shipping_computed;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'condition', 'shipping_computed', 'count'])
print(result_df)

   sponsored                 condition shipping_computed  count
0       true  Manufacturer refurbished              true     50
1       true        Seller refurbished              true     10
2       true                      Used              true      8
3       true                  Open box              true      3
4       true                       New              true      1
5      false        Seller refurbished              true    201
6      false                      Used              true    329
7      false                  Open box              true     88
8      false  Manufacturer refurbished              true    152
9      false                      Used             false    132
10     false                       New              true     95
11     false  Manufacturer refurbished             false      7
12     false                       New             false     19
13     false        Seller refurbished             false      3
14     false                  Open box  

In [41]:
# sponsored or not

mycursor.execute("select sponsored_or_not, count(*) as count from ebay_items group by sponsored_or_not;")
result = mycursor.fetchall()
result_df = pd.DataFrame(result, columns=['sponsored', 'count'])
print(result_df)

  sponsored  count
0      true     72
1     false   1034


### g

g) Use the stats in (f) and tell in your own words, how sponsored and non-sponsored items appear to be different. Is there a variable/column that can be used to predict the sponsor/non-sponsor items. (You do NOT need to run any model or statistic calculations for this part. Make your judgment only by looking at the stats you printed in (f))

In [ ]:
# According to the calculation results above, we can see that the sponsored and non-sponsored items are obviously different 
# in seller score, shipping price and return policy. Sellers of sponsored items usually have higher score and lower shipping 
# price. And sellers of sponsored items usually allow return.
# Thus, we might use these variables to predict the sponsored/non-sponsored items.